In [2]:
!pip install wandb fasttext plotly

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.8/68.8 kB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.1/267.1 kB 13.7 MB/s eta 0:00:00
  Using cached pybind11-2.12.0-py3-none-any.whl (234 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 5.6 MB/s eta 0:00:00
  Created wheel for fasttext: filename=fasttext-0.9.2-cp310-cp310-linux_x86_64.whl size=4227139 sha256=67f06e50cc967bbd3d0861c7b804c34ed0825c875458adb0a55c878f5a74dd0d
  Stored in directory: /root/.cache/pip/wheels/a5/13/75/f811c84a8ab36eedbaef977a6a58a98990e8e0f1967f98f394
Successfully built fasttext


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import json
import numpy as np
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, accuracy_score
import torch
from torch.utils.data import Dataset, DataLoader
from torch import nn
import fasttext.util
import plotly.express as px
import sys
import wandb
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence
from torch.nn.functional import sigmoid
import pdb
from torch.optim import Adam
import random as rnd
import nltk
nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [5]:
def set_seed(seed):
    np.random.seed(seed)
    rnd.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # Se stai usando GPU

# Imposta il seed per avere riproducibilità
seed = 42
set_seed(seed)

In [6]:
# "neutrale", "odio" aaf831dabc88d936d4e6b439b798bb4cb42814ea

ROOT_PATH = "/content/drive/MyDrive/uni/nlp/nlp2024-hw1-b"



In [7]:
wandb.login(key='aaf831dabc88d936d4e6b439b798bb4cb42814ea')

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [8]:


class HaSpeeDe_Dataset(Dataset):


    def __init__(self, data_path: str, data: list[tuple[list, int]]=None, use_embeddings: bool=False, stopwords_file_path: str="", device="cpu") -> None:
        self.device = device
        if data is not None:
            self.data = data
        else:
            stopwords = []
            if stopwords_file_path != "":
                with open(stopwords_file_path, 'r', encoding="UTF8") as f:
                    stopwords = f.readlines() #controllare carattere di andare a capo
            if use_embeddings:
                fasttext.util.download_model('it', if_exists='ignore')
                embeddings = fasttext.load_model('cc.it.300.bin')
            self.data = []
            with open(data_path, 'r', encoding="UTF8") as f:
                for line in f:
                    item = json.loads(line)
                    sentence = nltk.word_tokenize(item['text'], language='italian')
                    filtered_sentence = []
                    i=0
                    while i < len(sentence):
                        word = sentence[i]
                        if (word == "#" or word == "@") and i+1 < len(sentence):
                            word = word + sentence[i+1]
                            i+=1
                        if stopwords_file_path == "" or word not in stopwords:
                            filtered_sentence.append(word)
                        i+=1
                    if use_embeddings:
                        embedded_sentence = []
                        for word in sentence:
                            embedded_sentence.append(embeddings.get_word_vector(word))
                        sentence = embedded_sentence
                    self.data.append((sentence, item['label']))

    def __len__(self) -> int:
        return len(self.data)

    def __getitem__(self, idx: int) -> tuple[list, int]:
        return self.data[idx]

    def split(self, prc: float) -> list[tuple[list, int]]:
        validation_size = int(prc * len(self.data))
        train_size = len(self.data) - validation_size
        validation_data, self.data = torch.utils.data.random_split(self.data, [validation_size, train_size])
        return validation_data


    def collate(self, batch: list[tuple[list, int]]) -> tuple[torch.Tensor, torch.Tensor, torch.Tensor]:
        texts, labels = zip(*batch)
        lens = [len(text) for text in texts]
        texts = pad_sequence([torch.tensor(text) for text in texts], batch_first=True)
        return texts.to(self.device), torch.tensor(labels, dtype=torch.float).to(self.device), torch.tensor(lens).to(self.device)

    def get_dataloader(self, batch_size: int, shuffle: bool) -> DataLoader:
        return DataLoader(self, batch_size=batch_size, shuffle=shuffle, collate_fn=self.collate)


    def print_data_analysis(self):
        y= [0,0]
        for el in self.data:
            if el[1] == 1:
                y[1] += 1
            else:
                y[0] += 1
        self.neutral_count = y[0]
        self.hateful_count = y[1]
        fig = px.bar(x=["neutrale", "odio"], y=y)
        fig.show()






In [9]:
# Function to print a progress bar
def print_progress_bar(percentuale: float, lunghezza_barra: int = 30, text: str="") -> None:
    blocchi_compilati = int(lunghezza_barra * percentuale)
    barra = "[" + "=" * (blocchi_compilati - 1) + ">" + " " * (lunghezza_barra - blocchi_compilati) + "]"
    sys.stdout.write(f"\r{barra} {percentuale * 100:.2f}% complete " + text)
    sys.stdout.flush()

In [10]:


class Trainer():

    def __init__(self, model,train_dataloader, validation_dataloader, optimizer, loss_function, device, test_dataloader=None):
        self.model = model.to(device)
        self.train_dataloader = train_dataloader
        self.validation_dataloader = validation_dataloader
        self.optimizer = optimizer
        self.loss_function = loss_function
        self.device = device
        self.test_dataloader = test_dataloader


    @staticmethod
    def evaluation_parameters(y_true, y_pred):
        cm = confusion_matrix(y_true, y_pred).ravel()
        precision = precision_score(y_true, y_pred)
        recall = recall_score(y_true, y_pred)
        f1 = f1_score(y_true, y_pred)
        accuracy = accuracy_score(y_true, y_pred)
        return cm, precision, recall, f1, accuracy


    def train(self, epochs: int, use_wandb: bool = False, config: dict = {}, name: str=""):
        if use_wandb:
            wandb.init(
                # Set the project where this run will be logged
                project="nlp-hw-1b",
                name=name,
                # Track hyperparameters and run metadata
                config=config
            )
        for epoch in range(epochs):
            self.model.train()  # Set the model to training mode
            total_loss = 0
            #breakpoint()
            for i, batch in enumerate(self.train_dataloader):
                print_progress_bar(i / len(self.train_dataloader), text=f"| training epoch {epoch}")
                # Get the inputs and targets from the batch
                inputs, targets, lens = batch
                # Zero the gradients
                self.optimizer.zero_grad()
                # Forward pass
                outputs = self.model((inputs, lens))
                #breakpoint()
                # Compute loss
                loss = self.loss_function(outputs, targets)
                # Backward pass and optimize
                loss.backward()
                self.optimizer.step()
                # Accumulate the total loss
                total_loss += loss.item()

            # Print the average loss for this epoch
            validation_loss, precision, recall, f1, accuracy = self.validate(use_wandb)
            if use_wandb:
                wandb.log({"validation_loss": validation_loss,
                      "precision": precision,
                      "recall": recall,
                      "f1": f1,
                      "accuracy": accuracy,
                      "train_loss": total_loss / len(self.train_dataloader)})

        if use_wandb:
            wandb.finish()



    def validate(self, use_wandb: bool = False, test=False):
        dataloader = self.test_dataloader if test else self.validation_dataloader
        if dataloader is None:
            print("empty dataloader!")
            exit(1)
        self.model.eval()  # Set the model to evaluation mode
        total_loss = 0
        all_predictions = torch.tensor([], device=self.device)
        all_targets = torch.tensor([], device=self.device)
        with torch.no_grad():  # Do not calculate gradients
            for i, batch in enumerate(self.validation_dataloader):
                print_progress_bar(i / len(dataloader), text="| validation")
                # Get the inputs and targets from the batch
                inputs, targets, lens = batch
                # Forward pass
                outputs = self.model((inputs,lens))
                # Compute loss
                loss = self.loss_function(outputs, targets)
                # Accumulate the total loss
                total_loss += loss.item()
                # Store predictions and targets
                all_predictions = torch.cat((all_predictions, outputs.squeeze().round()))
                all_targets = torch.cat((all_targets, targets))
        validation_loss = total_loss / len(self.validation_dataloader)
        precision = precision_score(all_targets, all_predictions)
        recall = recall_score(all_targets, all_predictions)
        f1 = f1_score(all_targets, all_predictions)
        accuracy = accuracy_score(all_targets, all_predictions)
        return validation_loss, precision, recall, f1, accuracy




In [ ]:


class BaselineStratifiedModel(nn.Module):

    def __init__(self, len0, len1):
        super(BaselineStratifiedModel, self).__init__()
        self.p = len0/(len0+len1)

    def forward(self, x):
        return torch.tensor([0 if np.random.rand() < self.p else 1 for _ in range(x[0].shape[0])], dtype=torch.float)



class BaselineSimpleModel(nn.Module):

    def __init__(self, input_size, output_size):
        super(BaselineSimpleModel, self).__init__()
        self.linear = nn.Linear(input_size, output_size)
        # for param in self.linear.parameters():
        #     param.requires_grad = True

    def forward(self, x):
        #breakpoint()
        seq, lens = x
        packed = pack_padded_sequence(seq, lens, batch_first=True, enforce_sorted=False)
        data = self.linear(packed.data).squeeze()
        data = sigmoid(data)
        packed = torch.nn.utils.rnn.PackedSequence(data, packed.batch_sizes, packed.sorted_indices, packed.unsorted_indices)
        seq, lens = pad_packed_sequence(packed, batch_first=True)
        seq_mean = seq.sum(dim=1) / lens.float() #torch.tensor([sum(el)/len(el) for el in seq])
        #breakpoint()
        return seq_mean





In [11]:

class ClassifierModule(nn.Module):
    def __init__(self,sizes, dropout=0):
        super(ClassifierModule,self).__init__()
        sequence = []
        for i in range(len(sizes)-1):
            sequence += [nn.Linear(sizes[i],sizes[i+1]), nn.ReLU(), nn.Dropout(dropout)]
        self.classifier = nn.Sequential(*sequence)


    def forward(self,inputs):
        return self.classifier(inputs)





In [12]:

class HateDetectionModule(nn.Module):

    def __init__(self,classifier, input_size, hidden_size, dropout=0):
        super(HateDetectionModule,self).__init__()
        self.classifier = classifier
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True, dropout=dropout, bidirectional=True)
        self.sigmoid = nn.Sigmoid()


    def forward(self,x):

        input,lengths = x

        padded = pack_padded_sequence(input,lengths, batch_first=True,enforce_sorted=False)

        o, (h, c) = self.lstm(padded)

        out = pad_packed_sequence(o,batch_first=True)[0]

        output = self.classifier(out)
        return  self.sigmoid(output)




In [12]:

train_dataset = HaSpeeDe_Dataset(ROOT_PATH + "/train-taskA.jsonl", use_embeddings=True)
train_dataset.print_data_analysis()

In [ ]:
val_data = train_dataset.split(0.2)
val_dataset = HaSpeeDe_Dataset("", data=val_data)
train_loader = train_dataset.get_dataloader(64, True)
val_loader = val_dataset.get_dataloader(64, True)
test_dataset = HaSpeeDe_Dataset(ROOT_PATH + "/test-news-taskA.jsonl", use_embeddings=True) #/test-tweets-taskA.jsonl
test_loader = test_dataset.get_dataloader(64, True)

In [ ]:
model = BaselineSimpleModel(300,1)


trainer = Trainer(model,train_loader, val_loader, Adam(model.parameters(), lr=2e-1, weight_decay=1e-4), nn.BCELoss(), "cpu",test_dataloader=test_loader)

In [ ]:

classifier = ClassifierModule([2048, 1024, 1])
hate_detector = HateDetectionModule(classifier, 300, 1024)


hate_trainer = Trainer(hate_detector,train_loader, val_loader, Adam(model.parameters()), nn.BCELoss(), "cuda",test_dataloader=test_loader)

In [ ]:

baseline_model = BaselineStratifiedModel(train_dataset.neutral_count, train_dataset.hateful_count)


trainer_baseline = Trainer(baseline_model,train_loader, val_loader, Adam(model.parameters()), nn.BCELoss(), "cpu",test_dataloader=test_loader)




In [ ]:

validation_loss, precision, recall, f1, accuracy = trainer_baseline.validate()



[===========================>  ] 95.45% complete

In [ ]:


trainer.train(30, use_wandb=True , name="simple-baseline-lr0.2")


[==========================>   ] 90.70% complete | training epoch 9

KeyboardInterrupt: 

In [ ]:


print("Validation Loss:", validation_loss)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("Accuracy:", accuracy)

Validation Loss: 46.73604254289107
Precision: 0.4165103189493433
Recall: 0.4014466546112116
F1 Score: 0.4088397790055248
Accuracy: 0.5303584491587418


In [ ]:
wandb.finish()

accuracy,▁▆▇▅█▃▅▄▅
f1,▁██▅▇▄▅▃▅
precision,▆▁▅▇█▄▆█▅
recall,▁█▇▄▆▄▅▃▅
train_loss,█▁▂▁▁▁▁▁▁
validation_loss,▇█▆▁▃▂▁▂▂
accuracy,0.69934
f1,0.5776
precision,0.66118
recall,0.51277
train_loss,0.57573
